In [2]:
data = ('LR',
 {'11A': ('11B', 'XXX'),
  '11B': ('XXX', '11Z'),
  '11Z': ('11B', 'XXX'),
  '22A': ('22B', 'XXX'),
  '22B': ('22C', '22C'),
  '22C': ('22Z', '22Z'),
  '22Z': ('22B', '22B'),
  'XXX': ('XXX', 'XXX')})

In [27]:
def find_pattern(path):
    offset = 0
    while offset < len(path):
        if (len(path) - offset) % 3 == 0:
            pattern_len = (len(path) - offset) // 3
            first = path[offset:pattern_len + offset]
            second = path[pattern_len + offset:pattern_len * 2 + offset]
            third = path[pattern_len * 2 + offset:pattern_len * 3 + offset]
            if first == second == third:
                return offset, first
        offset += 1
    return offset, []


for test_path in [
    ["AAA", "BBB", "AAA", "BBB", "AAA", "BBB"],
    ["AAA", "AAA", "BBB", "AAA", "BBB", "AAA", "BBB"],
    ["AAA", "BBB", "CCC", "AAA", "CCC", "BBB", "AAA", "CCC", "BBB", "AAA", "CCC", "BBB"],
]:
    print("PATH", test_path)
    print("PATTERN", find_pattern(test_path))

PATH ['AAA', 'BBB', 'AAA', 'BBB', 'AAA', 'BBB']
PATTERN (0, ['AAA', 'BBB'])
PATH ['AAA', 'AAA', 'BBB', 'AAA', 'BBB', 'AAA', 'BBB']
PATTERN (1, ['AAA', 'BBB'])
PATH ['AAA', 'BBB', 'CCC', 'AAA', 'CCC', 'BBB', 'AAA', 'CCC', 'BBB', 'AAA', 'CCC', 'BBB']
PATTERN (3, ['AAA', 'CCC', 'BBB'])


In [36]:
def convert_to_pattern(path):
    offset, pattern = find_pattern(path)
    end_search_path = pattern if len(pattern) > 0 else path
    for i in range(len(end_search_path)):
        if end_search_path[i].endswith("Z"):
            return offset, len(pattern), i
    return len(path), 0, 0
        

for test_path in [    
    ["AAA", "BBZ", "AAA", "BBZ", "AAA", "BBZ"],
    ["AAA", "AAZ", "BBB", "AAZ", "BBB", "AAZZ", "BBB"],
    ["AAA", "BBB", "CCC", "AAA", "CCZ", "BBB", "AAA", "CCZ", "BBB", "AAA", "CCZ", "BBB"],
]:
    print("PATH", test_path)
    print("PATTERN", convert_to_pattern(test_path))

PATH ['AAA', 'BBZ', 'AAA', 'BBZ', 'AAA', 'BBZ']
PATTERN (0, 2, 1)
PATH ['AAA', 'AAZ', 'BBB', 'AAZ', 'BBB', 'AAZZ', 'BBB']
PATTERN (7, 0, 1)
PATH ['AAA', 'BBB', 'CCC', 'AAA', 'CCZ', 'BBB', 'AAA', 'CCZ', 'BBB', 'AAA', 'CCZ', 'BBB']
PATTERN (3, 3, 1)


In [45]:
def get_path_pattern(start_point, data):
    instructions, directions = data
    path = [start_point]
    pattern = (0, 0, 0)
    step = 0
    while pattern[1] == 0:
        print("PATH", path)
        current_point = path[-1]
        direction = instructions[step % len(instructions)]
        next_point = directions[current_point][int(direction == "R")]
        path.append(next_point)
        step += 1
        pattern = convert_to_pattern(path)
    return pattern

get_path_pattern("11A", data)

PATH ['11A']
PATH ['11A', '11B']
PATH ['11A', '11B', '11Z']
PATH ['11A', '11B', '11Z', '11B']
PATH ['11A', '11B', '11Z', '11B', '11Z']
PATH ['11A', '11B', '11Z', '11B', '11Z', '11B']


(1, 2, 1)

In [46]:
from math import lcm


def part2(data):
    instructions, directions = data
    currents = [key for key in directions.keys() if key[-1] == 'A']
    patterns = [get_path_pattern(current, data) for current in currents]
    print(patterns)

    return lcm(*[pattern[1] for pattern in patterns])


part2(data)

PATH ['11A']
PATH ['11A', '11B']
PATH ['11A', '11B', '11Z']
PATH ['11A', '11B', '11Z', '11B']
PATH ['11A', '11B', '11Z', '11B', '11Z']
PATH ['11A', '11B', '11Z', '11B', '11Z', '11B']
PATH ['22A']
PATH ['22A', '22B']
PATH ['22A', '22B', '22C']
PATH ['22A', '22B', '22C', '22Z']
PATH ['22A', '22B', '22C', '22Z', '22B']
PATH ['22A', '22B', '22C', '22Z', '22B', '22C']
PATH ['22A', '22B', '22C', '22Z', '22B', '22C', '22Z']
PATH ['22A', '22B', '22C', '22Z', '22B', '22C', '22Z', '22B']
PATH ['22A', '22B', '22C', '22Z', '22B', '22C', '22Z', '22B', '22C']
[(1, 2, 1), (1, 3, 2)]


6

In [6]:
from math import lcm


def part2v2(data):
    instructions, directions = data
    currents = [key for key in directions.keys() if key[-1] == 'A']
    shortest_paths = [0 for _ in currents]
    step = 0
    while any([shortest_path == 0 for shortest_path in shortest_paths]):
        direction = instructions[step % len(instructions)]
        currents = [directions[current][int(direction == "R")] for current in currents]
        shortest_paths = [
            step + 1 if shortest_path == 0 and current.endswith("Z") else shortest_path 
            for shortest_path, current in zip(shortest_paths, currents)
        ]
        step += 1
        print("CURRENTS", currents)
        print("SHORTEST PATHS", shortest_paths)
    return lcm(*shortest_paths)


part2v2(data)

CURRENTS ['11B', '22B']
SHORTEST PATHS [0, 0]
CURRENTS ['11Z', '22C']
SHORTEST PATHS [2, 0]
CURRENTS ['11B', '22Z']
SHORTEST PATHS [2, 3]


6

In [7]:
def parse_data(data_raw):
    lines = data_raw.splitlines()
    instructions = lines[0]
    directions = {}
    for line in lines[2:]:
        source, dest_raw = line.split(' = ')
        dest_left, dest_right = dest_raw[1:-1].split(', ')
        directions[source] = (dest_left, dest_right)
    return instructions, directions


with open('input.txt') as f:
    real_data = parse_data(f.read())

In [8]:
part2v2(real_data)

CURRENTS ['FMP', 'SXC', 'KVC', 'DFD', 'SHR', 'LBP']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['BSP', 'QRX', 'NGT', 'MVH', 'GQQ', 'MKR']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['RSM', 'RQX', 'FHF', 'VKM', 'THH', 'JPG']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['JGJ', 'KSX', 'RTX', 'GJF', 'BGN', 'KLN']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['RPK', 'SRM', 'VML', 'GLG', 'SMG', 'NLS']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['CMR', 'LFG', 'GKX', 'GXP', 'TVS', 'BTD']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['RLS', 'CSD', 'GTX', 'PCV', 'JNS', 'CRF']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['NVC', 'TMH', 'GFT', 'HTL', 'HSR', 'BRS']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['KRJ', 'RCJ', 'GCD', 'HKD', 'FFK', 'SRC']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['NHX', 'RXK', 'GCH', 'BLG', 'QXF', 'TPK']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['RGS', 'XJK', 'KXL', 'VTL', 'DGF', 'SRF']
SHORTEST PATHS [0, 0, 0, 0, 0, 0]
CURRENTS ['JKJ', 'BFF', 'KNH', 'KLC', 'BDS', 'DLG']
SH

15299095336639